In [1]:
import os

%pwd

'c:\\Users\\satis\\Kidney-Disease-Classification-DP-Project\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\satis\\Kidney-Disease-Classification-DP-Project'

In [3]:
import mlflow
from urllib.parse import urlparse

In [4]:
# import dagshub
# dagshub.init(repo_owner='sateesh-akula', repo_name='Kidney-Disease-Classification-DP-Project', mlflow=True)



In [5]:
# mlflow.set_registry_uri('https://dagshub.com/sateesh-akula/Kidney-Disease-Classification-DP-Project.mlflow')
# t = urlparse(mlflow.get_tracking_uri()).scheme
# print(t)

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
import dagshub
dagshub.init(repo_owner='sateesh-akula', repo_name='Kidney-Disease-Classification-DP-Project', mlflow=True)


Accessing as sateesh-akula

[2024-07-04 20:17:13,644: INFO: helpers: Accessing as sateesh-akula]


Initialized MLflow to track repo "sateesh-akula/Kidney-Disease-Classification-DP-Project"

[2024-07-04 20:17:14,512: INFO: helpers: Initialized MLflow to track repo "sateesh-akula/Kidney-Disease-Classification-DP-Project"]


Repository sateesh-akula/Kidney-Disease-Classification-DP-Project initialized!

[2024-07-04 20:17:14,512: INFO: helpers: Repository sateesh-akula/Kidney-Disease-Classification-DP-Project initialized!]


In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET",
            mlflow_uri="https://dagshub.com/sateesh-akula/Kidney-Disease-Classification-DP-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-04 20:19:05,282: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-04 20:19:05,286: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-04 20:19:05,288: INFO: common: Created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 93s 671ms/step - loss: 2.4700 - accuracy: 0.6620
[2024-07-04 20:20:38,838: INFO: common: json file saved at: {path}]


2024/07/04 20:20:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-04 20:20:41,487: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\satis\AppData\Local\Temp\tmp35nzck40\model\data\model\assets
[2024-07-04 20:20:41,862: INFO: builder_impl: Assets written to: C:\Users\satis\AppData\Local\Temp\tmp35nzck40\model\data\model\assets]


c:\Users\satis\anaconda3\envs\kidneyENV\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/07/04 20:21:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
